# Traing

In [ ]:
!git clone https://github.com/tloen/alpaca-lora

In [6]:
!cd alpaca-lora && pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-8tjavhi9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-8tjavhi9
  Resolved https://github.com/huggingface/peft.git to commit 382b178911edff38c1ff619bbac2ba556bd2276b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-84yxngl_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-84yxngl_
  Resolved https://github.com/huggingface/transformers.git to commit 12d51db243a00726a548a43cc333390ebae731e3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml)

# 评估

In [1]:
import torch
from peft import PeftModel
import transformers
import gradio as gr

assert (
    "LlamaTokenizer" in transformers._import_structure["models.llama"]
), "LLaMA is now in HuggingFace's main branch.\nPlease reinstall it: pip uninstall transformers && pip install git+https://github.com/huggingface/transformers.git"
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig

tokenizer = LlamaTokenizer.from_pretrained("/input0")

BASE_MODEL = "/input0"
LORA_WEIGHTS = "./codegen"

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

try:
    if torch.backends.mps.is_available():
        device = "mps"
except:
    pass

if device == "cuda":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        load_in_8bit=False,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    model = PeftModel.from_pretrained(
        model, LORA_WEIGHTS, torch_dtype=torch.float16, force_download=True
    )
elif device == "mps":
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
        torch_dtype=torch.float16,
    )
else:
    model = LlamaForCausalLM.from_pretrained(
        BASE_MODEL, device_map={"": device}, low_cpu_mem_usage=True
    )
    model = PeftModel.from_pretrained(
        model,
        LORA_WEIGHTS,
        device_map={"": device},
    )


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""

if device != "cpu":
    model.half()
model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=128,
    **kwargs,
):
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        **kwargs,
    )
    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /usr/local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('443'), PosixPath('//10.111.0.1'), PosixPath('tcp')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('phodal/jobs/oyru2aygme52')}
  warn(msg)
/usr/local/lib/python3.8/site-packages/bitsa

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [2]:
print(evaluate("Implement the method recursivelyDeleteFile", """
FileUtils:
- recursivelyDeleteFile(File, Function<String, Boolean>): boolean
""", 0.1, 0.75, 40, 4, 512))

/**
 * @since 1.4.0
 */
public class FileUtils {

    public static boolean recursivelyDeleteFile(File file, Function<String, Boolean> callback) {
        if (file.isDirectory()) {
            File[] files = file.listFiles();
            if (files != null) {
                for (File f : files) {
                    recursivelyDeleteFile(f, callback);
                }
            }
        }
        if (file.delete()) {
            return true;
        }
        if (callback != null) {
            callback.apply(file.getAbsolutePath());
        }
        return false;
    }
}

### Input:
/**
 * @since 1.4.0
 */
public class FileUtils {

    public static boolean recursivelyDeleteFile(File file, Function<String, Boolean> callback) {
        if (file.isDirectory()) {
            File[] files = file.listFiles();
            if (files != null) {
                for (File f : files) {
                    recursivelyDeleteFile(f, callback);
                }
            }
        }
        

In [4]:
print(evaluate("Implement the method chmod", """
PathUtils:
- methods:chmod(String, int)
""", 0.1, 0.75, 40, 4, 512))

/*
 * Copyright 2015-2023 the original author or authors.
 *
 * All rights reserved. This program and the accompanying materials are
 * made available under the terms of the Eclipse Public License v2.0 which
 * accompanies this distribution and is available at
 *
 * https://www.eclipse.org/legal/epl-v20.html
 */
/**
 * @since 1.0
 */
public class PathUtils {

    private PathUtils() {
    }

    public static void chmod(String path, int mode) {
        File file = new File(path);
        if (file.isDirectory()) {
            FileUtils.chmod(file, mode);
        } else {
            FileUtils.chmod(new File(path), mode);
        }
    }

    public static void chmod(File file, int mode) {
        if (file.isDirectory()) {
            FileUtils.chmod(file, mode);
        } else {
            FileUtils.chmod(new File(file.getParent(), file.getName()), mode);
        }
    }

    public static void chmod(Path path, int mode) {
        if (path.isAbsolute()) {
            FileUtils.chmod(pa

In [7]:
print(evaluate("Implement the method publish", """
ReleaseService(final UserInfoHolder userInfoHolder, final AdminServiceAPI.ReleaseAPI releaseAPI):
- method: publish(NamespaceReleaseModel): ReleaseDTO
""", 0.1, 0.75, 40, 4, 512))

/*
 * Copyright 2015-2023 the original author or authors.
 *
 * All rights reserved. This program and the accompanying materials are
 * made available under the terms of the Eclipse Public License v2.0 which
 * accompanies this distribution and is available at
 *
 * https://www.eclipse.org/legal/epl-v20.html
 */
@Service
public class ReleaseService {

    private final UserInfoHolder userInfoHolder;

    private final AdminServiceAPI.ReleaseAPI releaseAPI;

    @Autowired
    public ReleaseService(final UserInfoHolder userInfoHolder, final AdminServiceAPI.ReleaseAPI releaseAPI) {
        this.userInfoHolder = userInfoHolder;
        this.releaseAPI = releaseAPI;
    }

    public ReleaseDTO publish(final NamespaceReleaseModel namespaceReleaseModel) {
        final String username = userInfoHolder.getUsername();
        final String password = userInfoHolder.getPassword();
        final ReleaseDTO releaseDTO = releaseAPI.publish(username, password, namespaceReleaseModel);
        return

In [9]:
print(evaluate("Implement the method check", """
ConfigFileUtils:
- method: check(MultipartFile): void, checkEmpty(MultipartFile): void, checkFormat(String)
""", 0.1, 0.75, 40, 4, 512))

/*
 * Copyright 2015-2023 the original author or authors.
 *
 * All rights reserved. This program and the accompanying materials are
 * made available under the terms of the Eclipse Public License v2.0 which
 * accompanies this distribution and is available at
 *
 * https://www.eclipse.org/legal/epl-v20.html
 */
/**
 * @since 2.3
 */
public class ConfigFileUtils {

    private ConfigFileUtils() {
    }

    public static void check(MultipartFile file) {
        if (file == null) {
            throw new IllegalArgumentException("File is null");
        }
        if (file.getSize() <= 0) {
            throw new IllegalArgumentException("File is empty");
        }
        if (file.getSize() > MAX_FILE_SIZE) {
            throw new IllegalArgumentException("File is too large");
        }
        if (file.getSize() < MIN_FILE_SIZE) {
            throw new IllegalArgumentException("File is too small");
        }
        if (file.getSize() > MAX_FILE_SIZE) {
            throw new IllegalArgum

In [10]:
print(evaluate("Implement the method findApps", """
AppController(
      final UserInfoHolder userInfoHolder,
      final AppService appService,
      final PortalSettings portalSettings,
      final ApplicationEventPublisher publisher,
      final RolePermissionService rolePermissionService,
      final RoleInitializationService roleInitializationService,
      final AdditionalUserInfoEnrichService additionalUserInfoEnrichService):
- method: findApps(String): List<App>
""", 0.1, 0.75, 40, 4, 512))

@RestController
@RequestMapping("/apps")
public class AppController {

    private final UserInfoHolder userInfoHolder;

    private final AppService appService;

    private final PortalSettings portalSettings;

    private final ApplicationEventPublisher publisher;

    private final RolePermissionService rolePermissionService;

    private final RoleInitializationService roleInitializationService;

    private final AdditionalUserInfoEnrichService additionalUserInfoEnrichService;

    @Autowired
    public AppController(final UserInfoHolder userInfoHolder, final AppService appService, final PortalSettings portalSettings, final ApplicationEventPublisher publisher,
        final RolePermissionService rolePermissionService, final RoleInitializationService roleInitializationService,
        final AdditionalUserInfoEnrichService additionalUserInfoEnrichService) {
        this.userInfoHolder = userInfoHolder;
        this.appService = appService;
        this.portalSettings = portalSettin

In [12]:
print(evaluate("Implement the method findApps", """
AppService(
      final UserInfoHolder userInfoHolder,
      final AdminServiceAPI.AppAPI appAPI,
      final AppRepository appRepository,
      final ClusterService clusterService,
      final AppNamespaceService appNamespaceService,
      final RoleInitializationService roleInitializationService,
      final RolePermissionService rolePermissionService,
      final FavoriteService favoriteService,
      final UserService userService):
- method: findAll(Pageable): PageDTO<App>
""", 0.1, 0.75, 40, 4, 512))

public class AppService {

    private final UserInfoHolder userInfoHolder;

    private final AdminServiceAPI.AppAPI appAPI;

    private final AppRepository appRepository;

    private final ClusterService clusterService;

    private final AppNamespaceService appNamespaceService;

    private final RoleInitializationService roleInitializationService;

    private final RolePermissionService rolePermissionService;

    private final FavoriteService favoriteService;

    private final UserService userService;

    public AppService(final UserInfoHolder userInfoHolder, final AdminServiceAPI.AppAPI appAPI, final AppRepository appRepository, final ClusterService clusterService, final AppNamespaceService appNamespaceService, final RoleInitializationService roleInitializationService, final RolePermissionService rolePermissionService, final FavoriteService favoriteService, final UserService userService) {
        this.userInfoHolder = userInfoHolder;
        this.appAPI = appAPI;
        th